In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
from sklearn.preprocessing import StandardScaler

import nltk
# nltk.download('wordnet')
#from nltk.stem import PorterStemmer, WordNetLemmatizer

from sklearn.feature_extraction.text import TfidfVectorizer  # Thay đổi ở đây
from sklearn.neighbors import KNeighborsRegressor
from math import sqrt

import re
import string
from sklearn.metrics.pairwise import cosine_similarity

from sklearn.metrics import mean_squared_error

In [2]:
# Đọc dữ liệu
items_df = pd.read_csv('./items.csv')
ratings_df = pd.read_csv('./ratings.csv')
users_df = pd.read_csv('./users.csv')

In [3]:
items_df["feature"] = items_df["feature"].str.lower()

def remove_tags(text):
    pattern = re.compile("<.*?>")
    return pattern.sub(r'', text)

items_df['feature'] = items_df['feature'].apply(remove_tags)
punctuation = string.punctuation

def remove_punc(text):
    return text.translate(str.maketrans('', '', punctuation))

items_df['feature'] = items_df['feature'].apply(remove_punc)

In [4]:
# Sử dụng TfidfVectorizer và tính cosine similarity trên cột 'feature'
tfidf_vectorizer = TfidfVectorizer(max_features=5000, stop_words="english")
vectors = tfidf_vectorizer.fit_transform(items_df["feature"]).toarray()
similarity = cosine_similarity(vectors)

In [5]:
# Lưu DataFrame thành file CSV
#vectors_df = pd.DataFrame(vectors)
#vectors_df.to_csv('vt.csv', index=False)

In [6]:
feature_names = tfidf_vectorizer.get_feature_names_out()
print("Feature names:", feature_names)
name = pd.DataFrame(feature_names)
name.to_csv('name.csv', index=False)

Feature names: ['03mm' '05' '05mm' ... 'zippers' 'zone' 'zones']


In [7]:
def get_top_similar_items(item_id, similarity_matrix, items_df, top_n=5):
    item_similarities = similarity_matrix[item_id]  
    similar_items = np.argsort(item_similarities)[::-1]  
    
    similar_items = similar_items[similar_items != item_id]
    
    similar_item_ids = items_df.iloc[similar_items[:top_n]]['itemId'].values
    similar_item_names = items_df.iloc[similar_items[:top_n]]['itemName'].values

    return similar_item_ids, similar_item_names

item_id = int(input("Nhập ItemId: "))
top_similar_item_ids, top_similar_item_names = get_top_similar_items(item_id, similarity, items_df)

print(f"Top 5 sản phẩm tương tự:")
for idx, (item_id, item_name) in enumerate(zip(top_similar_item_ids, top_similar_item_names)):
    print(f"{idx + 1}. itemId: {item_id}, {item_name}")


Top 5 sản phẩm tương tự:
1. itemId: 1285, PetSafe Easysport Harness
2. itemId: 3595, DEXDOG EZHarness, Dog Harness | On/Off Quick | Easy Step in | Walk Vest
3. itemId: 12757, 2 Hounds Design Freedom No-Pull Dog Harness, Adjustable Comfortable Control for Dog Walking, Made in USA (Leash Sold Separately) (5/8&quot;)
4. itemId: 11607, 2 Hounds Design Freedom No-Pull Dog Harness and Leash, Adjustable Comfortable Control for Dog Walking, Made in USA (1&quot;)
5. itemId: 1128, Kurgo Tru-Fit No Pull Dog Harness, Easy Walking Harness, Quick On and Off Harness With Pet Seat Belt Tether for Car


In [8]:
ratings_matrix = ratings_df.to_numpy()

def get_items_rated_by_user(rate_matrix, user_id):
    user_row_indices = np.where(rate_matrix[:, 0] == user_id + 1)[0]
    if len(user_row_indices) == 0:
        return np.array([]), np.array([])  

    item_ids = rate_matrix[user_row_indices, 1] - 1  
    scores = rate_matrix[user_row_indices, 2]
    return item_ids.astype(int), scores  

In [9]:
n_users = users_df.shape[0]
#print("Number of users:", n_users)
n_items = items_df.shape[0]
#print("Number of items:", n_items)

In [10]:
knn_models = [None] * n_users

for n in range(n_users):
    ids, scores = get_items_rated_by_user(ratings_matrix, n)

    if len(ids) == 0:
        knn_models[n] = None  # Không có đánh giá
        continue

    n_neighbors = min(5, len(ids))
    knn = KNeighborsRegressor(n_neighbors=n_neighbors, weights='uniform', metric='cosine')
    Xhat = vectors[ids.astype(int), :]  
    y = scores  
    knn.fit(Xhat, y)
    knn_models[n] = knn

In [11]:
# Dự đoán điểm đánh giá cho tất cả người dùng và item đã được đánh giá
Yhat = np.zeros((n_items, n_users))

for n in range(n_users):
    knn = knn_models[n]
    if knn is None:
        continue 

    ids, scores = get_items_rated_by_user(ratings_matrix, n)
    if len(ids) == 0:
        continue

    Xhat = vectors[ids, :] 
    y_pred = knn.predict(Xhat)

    Yhat[ids, n] = y_pred

In [12]:
def evaluate(Yhat, rates):
    y_true = []
    y_pred = []

    for n in range(n_users):
        ids, scores_truth = get_items_rated_by_user(rates, n)
        if len(ids) == 0:
            continue 

        scores_pred = Yhat[ids, n]
        y_true.extend(scores_truth)
        y_pred.extend(scores_pred)

    mae = np.mean(np.abs(np.array(y_true) - np.array(y_pred))) if len(y_true) > 0 else float('nan')
    rmse = mean_squared_error(y_true, y_pred, squared=False) if len(y_true) > 0 else float('nan')

    return mae, rmse

mae, rmse = evaluate(Yhat, ratings_matrix)
print("MAE for training:", mae)
print("RMSE for training:", rmse)

MAE for training: 0.5754489526513736
RMSE for training: 0.9042204573458988


In [13]:
def suggest_items_for_user(Yhat, user_id, items_df, n_recommendations=10):
    # Lấy điểm dự đoán của người dùng user_id
    user_ratings_pred = Yhat[:, user_id]
    # Lấy danh sách các sản phẩm mà người dùng đã đánh giá
    rated_item_ids, _ = get_items_rated_by_user(ratings_matrix, user_id)
    # Loại bỏ những sản phẩm đã được đánh giá
    unrated_items = [i for i in range(n_items) if i not in rated_item_ids]
    # Dự đoán điểm số cho các sản phẩm chưa được đánh giá
    unrated_scores = user_ratings_pred[unrated_items]
    # Chọn ra các sản phẩm có điểm số cao nhất
    top_item_indices = np.argsort(unrated_scores)[-n_recommendations:][::-1]
    # Lấy ID và tên các sản phẩm gợi ý
    top_item_ids = items_df.iloc[unrated_items].iloc[top_item_indices]['itemId'].values
    top_item_names = items_df.iloc[unrated_items].iloc[top_item_indices]['itemName'].values
    
    return top_item_ids, top_item_names

# Gợi ý 10 sản phẩm cho người dùng đầu tiên 
recommendations_ids, recommendations_names = suggest_items_for_user(Yhat, user_id=0, items_df=items_df, n_recommendations=10)

# In ra các sản phẩm được gợi ý
print("Sản phẩm gợi ý cho người dùng dầu tiên:")
for idx, (item_id, item_name) in enumerate(zip(recommendations_ids, recommendations_names)):
    print(f"{idx + 1}. itemId: {item_id}, Tên sản phẩm: {item_name}")

Sản phẩm gợi ý cho người dùng dầu tiên:
1. itemId: 13988, Tên sản phẩm: Whitehall Personalized Cast Metal Address Plaque - Custom House Number Sign - Arched Rectangle (12 x 7.25) Black - Silver Numbers
2. itemId: 4691, Tên sản phẩm: Systemic Granules .22% 4 lb.
3. itemId: 4702, Tên sản phẩm: Fluval External Filter
4. itemId: 4701, Tên sản phẩm: PetSpy 1100 Yards Remote Dog Training Shock Collar for Dogs with Beep, Vibration and Electric Shock, Rechargeable and Waterproof E-Collar Trainer
5. itemId: 4700, Tên sản phẩm: HDP JW Dogs in Action
6. itemId: 4699, Tên sản phẩm: Niteangel Wooden Small Animals Platform, Fan Shape Springboard for Cage
7. itemId: 4698, Tên sản phẩm: uCOLOR Watercolor Turquoise Case for iPhone SE 5S 5 Protective Hard PC + Soft TPU Tough Case for iPhone SE 5S 5
8. itemId: 4697, Tên sản phẩm: Penn Plax Stone Replica Aquarium Decoration Realistic Granite Look with Fish Hideaway 8 Piece Set
9. itemId: 4696, Tên sản phẩm: Lanyar Dog Reindeer Holiday Pet Clothes Sweater 